In [0]:


# input_path = (
#     "abfss://bronze@storageaccountadls01.dfs.core.windows.net/Sales/Customer/Customer.parquet"
# )

# df = spark.read.format("parquet").load(input_path)
# display(df)

In [0]:
%python
# from pyspark.sql.functions import from_utc_timestamp, date_format, col
# from pyspark.sql.types import TimestampType

# df = df.withColumn(
#     "ModifiedDate",
#     date_format(
#         from_utc_timestamp(
#             col("ModifiedDate").cast(TimestampType()),
#             "UTC"
#         ),
#         "yyyy-MM-dd"
#     )
# )
# display(df)

In [0]:

# Import required PySpark functions and types
from pyspark.sql.functions import (
from_utc_timestamp, date_format, col, trim, when, lower, regexp_replace, isnan, lit)
from pyspark.sql.types import TimestampType, IntegerType, FloatType
import re
# Function to convert column names to snake case for consistency and readability
def to_snake_case(name):
    name = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    name = re.sub("([a-z0-9])([A-Z])", r"\1_\2", name) # Convert Case to snake_case
    return name.lower().replace(" ", "_")

# Collect all table (folder) names under the Bronze path
table_name = []
for i in dbutils.fs.ls("abfss://bronze@storageaccountadls01.dfs.core.windows.net/Sales/"):

    table_name.append(i.name.replace("/", "")) # Clean up folder names.

# Loop through each table and apply transformations
for i in table_name:
# Construct the full path to the Parquet file
  input_path = f"abfss://bronze@storageaccountadls01.dfs.core.windows.net/Sales/{i}/{i}.parquet"
# Read the Parquet file into a Spark DataFrame
df = spark.read.format("parquet").load(input_path)
# Rename all columns to snake_case
for old_col in df.columns:
    df = df.withColumnRenamed(old_col, to_snake_case(old_col))
# Apply standard transformations
for col_name in df.columns:
    # Convert date/datetime columns to yyyy-MM-dd format
    if "date" in col_name:
        df = df.withColumn(col_name, date_format(from_utc_timestamp(col(col_name).cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
    
    # Trim whitespaces in string columns
    if str(df.schema[col_name].dataType) == "StringType":
        df = df.withColumn(col_name, trim(col(col_name)))
    
    # Replace null/NaN in string columns with "Unknown"
    if str(df.schema[col_name].dataType) == "StringType":
        df = df.withColumn(
            col_name,
            when(col(col_name).isNull() | isnan(col(col_name)), lit("Unknown")).otherwise(col(col_name))
        )
# Replace null in integer columns with 0
if str(df.schema[col_name].dataType) == "IntegerType":
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(0)).otherwise(col(col_name))
    )

# Replace null in float columns with 0.0
if str(df.schema[col_name].dataType) == "FloatType":
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(0.0)).otherwise(col(col_name))
    )

# Remove duplicate rows if any
df = df.dropDuplicates()
# df.display()
# Define output path in Silver container
output_path = f"abfss://silver@forsynapseaccount .dfs.core.windows.net/Sales/{i}/"
# Write the transformed DataFrame to Silver layer in Delta format
# df.write.format("delta").mode("overwrite").save(output_path)python
for i in table_name:
    try:
        print(f"Attempting to process table: {i}")
        # input_path = f"abfss://bronze@storageaccountadls01.dfs.core.windows.net/Sales/{i}/{i}.parquet"
        # df = spark.read.format("parquet").load(input_path)
        # ... (all your transformation code goes here) ...
        output_path = f"abfss://silver@forsynapseaccount.dfs.core.windows.net/Sales/{i}/"
        df.write.format("delta").mode("overwrite").save(output_path)
        print(f"Successfully processed table: {i}")
    except Exception as e:
        print(f"FAILED to process table: {i}")
        # This will print the exact technical error message to your notebook output
        print(e) 